In [15]:
# main function that sets up environments
# perform training loop

import envs
from buffer import ReplayBuffer
from maddpg import MADDPG
import torch
import numpy as np
from tensorboardX import SummaryWriter
import os
from utilities import transpose_list, transpose_to_tensor

# keep training awake
# from workspace_utils import keep_awake

# for saving gif
import imageio

def seeding(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)

def pre_process(entity, batchsize):
    processed_entity = []
    for j in range(3):
        list = []
        for i in range(batchsize):
            b = entity[i][j]
            list.append(b)
        c = torch.Tensor(list)
        processed_entity.append(c)
    return processed_entity



seeding()
# number of parallel agents
parallel_envs = 5
# number of training episodes.
# change this to higher number to experiment. say 30000.
number_of_episodes = 16
episode_length = 10
batchsize = 10
# how many episodes to save policy and gif
save_interval = 5000
# what is this ?
t = 0

# amplitude of OU noise
# this slowly decreases to 0
noise = 2
noise_reduction = 0.9999

# how many episodes before update
episode_per_update = 2 * parallel_envs

log_path = os.getcwd()+"/log"
model_dir= os.getcwd()+"/model_dir"

os.makedirs(model_dir, exist_ok=True)

torch.set_num_threads(parallel_envs)
# this may be a list of all environments
env = envs.make_parallel_env(parallel_envs)

# keep 5000 episodes worth of replay
buffer = ReplayBuffer(int(5000*episode_length))

maddpg = MADDPG()
logger = SummaryWriter(log_dir=log_path)
agent0_reward = []
agent1_reward = []
agent2_reward = []

for episode in range(0, number_of_episodes, parallel_envs):



    reward_this_episode = np.zeros((parallel_envs, 3))

    all_obs = env.reset()
    obs, obs_full = transpose_list(all_obs)

    frames = []
    tmax = 0




    for episode_t in range(episode_length):

        
        t += parallel_envs

        actions = maddpg.act(transpose_to_tensor(obs), noise=noise)
        noise *= noise_reduction
        # there are 4 actions per agent and 3 agents, total of 12 actions 
        actions_array = torch.stack(actions).detach().numpy()
        
        actions_for_env = np.rollaxis(actions_array, 1)

        next_obs, next_obs_full, rewards, dones, info = env.step(actions_for_env)

        transition = (obs, obs_full, actions_for_env, rewards, next_obs, next_obs_full, dones)

        buffer.push(transition)
            
        reward_this_episode += rewards

        obs, obs_full = next_obs, next_obs_full

    # update once after every episode_per_update
    if len(buffer) > 1:
        for a_i in range(3):
            samples = buffer.sample(5)
            maddpg.update(samples, a_i, logger)
        maddpg.update_targets() #soft update the target network towards the actual networks

WARN: Could not seed environment <MultiAgentEnv instance>
WARN: Could not seed environment <MultiAgentEnv instance>
WARN: Could not seed environment <MultiAgentEnv instance>
WARN: Could not seed environment <MultiAgentEnv instance>
WARN: Could not seed environment <MultiAgentEnv instance>


In [19]:
actions = maddpg.act(transpose_to_tensor(obs), noise=noise)
noise *= noise_reduction
# there are 4 actions per agent and 3 agents, total of 12 actions 
actions_array = torch.stack(actions).detach().numpy()

actions_for_env = np.rollaxis(actions_array, 1)

In [20]:
print(torch.stack(actions).shape)

torch.Size([3, 5, 2])


In [22]:
actions_for_env.shape

(5, 3, 2)

In [18]:
actions_for_env.shape

(5, 3, 2)

In [14]:
a = np.ones((1,2,3))

print(a.shape, np.rollaxis(a, 1).shape)

(1, 2, 3) (2, 1, 3)


In [5]:
obs.shape

(4, 3, 14)

In [6]:
obs_full.shape

(4, 14)

In [10]:
len(actions)

3

In [13]:
actions[1].shape

torch.Size([4, 2])

In [ ]:
actions = maddpg.act(transpose_to_tensor(obs), noise=noise)


actions_array = torch.stack(actions).detach().numpy()

actions_for_env = np.rollaxis(actions_array, 1)